# KAIST Safety Course Automatic Viewer

This notebook makes viewing the (quite annoying) lab safety videos automatic, so you don't have to click many times on the videos that you need to see to complete the course :D


# Debug

In [122]:
%load_ext autoreload
%autoreload 2

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

# from kaist.navigation.video import watch_lessons
from kaist.navigation.quiz import run_quiz
from kaist.navigation.login import portal_login
from kaist.navigation.utils import save_debug, check_education_end, alert_accept, click_all_until_done
from kaist.utils import ThreadWithException
import time

from kaist.navigation.webdriver import setup_webdriver
from kaist.utils import ThreadWithException



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [130]:
config = {  'username': 'botussi',
            'automatic_login': True,
                    'video_id': 0,
                    'playback_rate': 10,
                    'mute_video': True,
                    'answer_quiz': True,
                    'browser': 'Chrome',
                    'driver_path': '/usr/bin/chromedriver',
                    'target_webpage': 'https://safety.kaist.ac.kr/main/main.do',
                    'debug_path': 'debug.html'}

In [136]:
driver = setup_webdriver('chrome')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 106.0.5249
[WDM] - Get LATEST chromedriver version for 106.0.5249 google-chrome
[WDM] - Driver [/home/botu/.wdm/drivers/chromedriver/linux64/106.0.5249.61/chromedriver] found in cache


In [137]:
driver.get(config['target_webpage'])

# Login into KAIST
portal_login(driver, config)

# Maximize window and open safety course page
print('Waiting for the button to be visible... You may need to maximize the window')
driver.maximize_window()
WebDriverWait(driver, timeout=60, poll_frequency=1).until(EC.element_to_be_clickable((By.ID, 'eduGo')))
button = driver.find_element(by='id', value='eduGo') # click button to go to page
button.click()

# Close popup
windows = driver.window_handles
driver.switch_to.window(windows[1]) # switch to popup window

alert_accept(driver, timeout=10)

# WebDriverWait(driver, timeout=60, poll_frequency=1).until(EC.element_to_be_clickable((By.ID, 'main_popup_0_close')))
# button = driver.find_element(by='id', value='main_popup_0_close') # click button to go to page

# Close all popups
click_all_until_done(driver, by=By.XPATH, value="//*[contains(text(), '닫기')]")

# Open My Page and close all possible alerts
button = driver.find_element(by=By.XPATH, value="//*[contains(text(), '마이페이지')]")
button.click()
alert_accept(driver, timeout=5)

Automatic login
Authenticate login with your phone. Alternatively, you can use the opened browser to navigate.
Waiting for login... Will timeout after `timeout` seconds
Waiting for the button to be visible... You may need to maximize the window


In [58]:
def find_all_iframes(driver):
    iframes = driver.find_elements(by=By.XPATH, value="//iframe")
    for index, iframe in enumerate(iframes):
        print(index)
        # Your sweet business logic applied to iframe goes here.
        driver.switch_to.frame(index)
        find_all_iframes(driver)
        driver.switch_to.parent_frame()


find_all_iframes(driver)

0
1
2


In [59]:
driver.switch_to.parent_frame()

In [60]:
from bs4 import BeautifulSoup

# driver.switch_to.frame(1)

def save_pretty(driver, filename='debug.html'):
    with open(filename, 'w') as f:
        f.write(BeautifulSoup(driver.page_source).prettify())

save_pretty(driver)

In [61]:
# Click the play button
driver.switch_to.frame(1)
idx = 0 # this will be the video index
buttons = driver.find_elements(by=By.CLASS_NAME, value="play_btn")
print(len(buttons))
buttons[idx].click() # list of possible buttons!
content = buttons[idx].get_attribute("onclick")
print("Video name: ", content.split("'")[-2])


1
Video name:  신입생 및 신규채용자 안전교육


In [62]:
driver.switch_to.parent_frame()

In [40]:
def find_all_iframes(driver):
    iframes = driver.find_elements(by=By.XPATH, value="//iframe")
    for index, iframe in enumerate(iframes):
        print(index)
        # Your sweet business logic applied to iframe goes here.
        driver.switch_to.frame(index)
        find_all_iframes(driver)
        driver.switch_to.parent_frame()

driver.switch_to.parent_frame()
find_all_iframes(driver)

0
1
2


In [126]:
# driver.switch_to.frame(1)
driver.switch_to.parent_frame()

save_pretty(driver)

In [128]:
# Turn off sound
driver.switch_to.parent_frame()
no_sound_option = True


button = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpSound")
print(len(button))
if len(button) > 0 and no_sound_option:
    button[0].click()

0


In [129]:
# Go to parent frame and start playing the video
driver.switch_to.parent_frame()
#btn_WF_VpConNext
button = driver.find_element(by=By.CLASS_NAME, value="btn_WF_VpPlay")
print(button)
button.click()


<selenium.webdriver.remote.webelement.WebElement (session="d8df8a4a1006bd3affb911ec9e4fa673", element="75506d34-7eec-4986-9726-bc8255dbe738")>


In [77]:
# Play
button = driver.find_element(by=By.CLASS_NAME, value="btn_WF_VpPlay")
button.click()

## Videos

In [141]:
retries = 0
while True:
    try:
        driver.switch_to.parent_frame()
        button_play = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpPlay")
        button_stop = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpStop")
        button_quiz = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpConQuiz")
        if len(button_play) > 0 or len(button_stop) > 0:
            if len(button_quiz) > 0:
                try:
                    button_quiz[0].click()
                    print("For now, you will need to answer the quiz yourself! Perhaps I will automate it someday ;)")
                except:
                    print("Tried clicking quiz button, but failed. Human, please try!")
                    pass
                finally:
                    break

            # We have the video screen open
            # Check if the video is playing, if not play it
            button = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpSound")
            if len(button) > 0 and config['mute_video']:
                print("Turning off sound")
                button[0].click()
            if len(button_play) > 0:
                print("Playing video")
                button_play[0].click()
                driver.execute_script(f'''document.querySelector('video').playbackRate = {config['playback_speed']};''')
                
        # else:
        #     # we have the video screen closed
        #     # print("No video screen detected! Please open video first")
        #     pass

    except Exception as e:
        retries += 1
        # alert_accept(driver, timeout=3)
        WebDriverWait(driver, 10).until_not(EC.alert_is_present(), "Popup alert found, please deal with it human!")
        if retries > 10:
            print("Too many retries, giving up!\nException: ", e)
            save_debug(driver, config['debug_path'])
            print("==============\nOuch, an error occurred! =(\nReload Driver to restart!\n==============")
            break

    time.sleep(3)

MaxRetryError: HTTPConnectionPool(host='localhost', port=49989): Max retries exceeded with url: /session/b69dd09785e27c3dbfa1ed68bebeab22/alert/text (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdbc56fca90>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [135]:
# Main playing loop

playback_rate = 10 # apparently, more than 10x does not work

# Check if any button for play of stop is available
while True:
    try:
        button_play = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpPlay")
        button_stop = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpStop")
        button_quiz = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpConQuiz")
        if len(button_play) > 0 or len(button_stop) > 0:
            if len(button_quiz) > 0:
                try:
                    button_quiz[0].click()
                except:
                    print("Tried clicking quiz button, but failed. Human, please try!")
                    pass
                finally:
                    break

            # We have the video screen open
            # Check if the video is playing, if not play it
            button = driver.find_elements(by=By.CLASS_NAME, value="btn_WF_VpSound")
            if len(button) > 0 and no_sound_option:
                print("Turning off sound")
                button[0].click()
            if len(button_play) > 0:
                print("Playing video")
                button_play[0].click()
                driver.execute_script(f'''document.querySelector('video').playbackRate = {playback_rate};''')
                
        else:
            # we have the video screen closed
            print("Video screen closed! Please open video first")
            break
        time.sleep(1)
    except KeyboardInterrupt:
        break

MaxRetryError: HTTPConnectionPool(host='localhost', port=51981): Max retries exceeded with url: /session/d560043395f708f50cadc169a56a2a79/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fdbc5711f90>: Failed to establish a new connection: [Errno 111] Connection refused'))

## Quiz

In [116]:
driver.switch_to.parent_frame()
save_pretty(driver)

In [120]:
quizzes = driver.find_elements(by=By.CLASS_NAME, value="quiz_Q")
# print(quizzes)

In [ ]:
quizzes[0]